# Streaming

<img src="https://github.com/niladridutta1988/wf_agentic_ai_training/blob/main/Batch2/lca-langchainV1-essentials/python_colab/assets/LC_streaming.png?raw=1" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [1]:
!pip install -qU langchain-groq langgraph langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [3]:
from langchain.agents import create_agent
from langchain_groq import ChatGroq

# Initialize the Groq model
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)

In [4]:
agent = create_agent(
    #model="openai:gpt-5",
    model=llm,
    system_prompt="You are a full-stack comedian",
)

## No Streaming (invoke)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Here's one:

You know what's wild? We spend the first year of a child's life teaching them to walk and talk, and the rest of their lives telling them to shut up and sit down.


## values
You have seen this streaming mode in our examples so far.

In [6]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Here's one:

"I told my wife she was drawing her eyebrows too high. She looked surprised."

How was that? Did I make you groan?


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [7]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Here's a family-friendly poem for you:

There once was a family so bright,
Their laughter and love shone with delight.
They'd gather 'round, hand in hand,
And make memories that would forever stand.

Their house was a home, full of cheer,
Where hugs and kisses were always near.
Their table was filled with yummy treats,
And their hearts were full of love that skips beats.

The kids would play outside all day,
Chasing butterflies in a sunny way.
Their parents would watch with a smile so wide,
Proud of the little ones, side by side.

As the sun sets and the day grows old,
The family would snuggle up, young and bold.
They'd share stories and secrets, and dreams so bright,
And fill each other's hearts with love and light.

So here's to the family, a shining star,
A bundle of love that goes near and far.
May their laughter and joy be contagious and free,
A family's love, a treasure to see!

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [8]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    #model="openai:gpt-5-mini",
    model=llm,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='150e60fa-00de-4a45-abdb-4c67b0239383')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='150e60fa-00de-4a45-abdb-4c67b0239383'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'q5ryfaf3n', 'function': {'arguments': '{"city":"SF"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 219, 'total_tokens': 233, 'completion_time': 0.034433469, 'completion_tokens_details': None, 'prompt_time': 0.017501791, 'prompt_tokens_details': None, 'queue_time': 0.068579501, 'total_time': 0.05193526}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9ca2574dca', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bb309-1061-7fc2-a329-6dc

In [9]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: SF')
('custom', 'Acquired data for city: SF')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [10]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: SF
Acquired data for city: SF
